Experiment to fix intrablock splits using slice to slice IoU tracking

In [ ]:
from cerebellum.data_prep.seg_prep import *

import json

with open('data_locs.json') as f:
	data_locs = json.load(f)
resolution = (30, 48, 48)
bbox = data_locs["gt"]["8nm-bbox"] # global bbox
affinity_offset = data_locs["aff-offset"] # affinity offset along z-axis
block_size = 60
wz_thresh = 0.5

In [2]:
block_id = 0
zz = block_id*block_size + affinity_offset
block_name = "waterz%.2f-48nm-crop2gt-%04d"%(wz_thresh, zz)
block = SegPrep(block_name, resolution)
block.read_internal(stage="filtered")
block.read_bboxes(stage="filtered") # Warning! If objects are relabeled, load relabeled-bboxes.json
block_seg = block.data
print block_seg.shape
bbox_dict = block.bbox_dict
print (len(bbox_dict.keys()))

(60, 540, 489)
5738


In [9]:
from cerebellum.error_correction.slice_stitch import slice2slice_iou_calc

def block_track(block_seg, bbox_dict, iou_thresh, zstride=1):
    
    def track_and_relabel(sslice, tslice, obj_id, block_seg, bbox_dict, iou_thresh):
        int_ids, ints, unions = slice2slice_iou_calc(sslice, tslice, obj_id)
        t_id = int_ids[0]
        iou = float(ints[0])/unions[0]
        track_success = iou>iou_thresh
        if track_success:
            print "Trakced to object: %d"%(t_id)
            bbox = bbox_dict[t_id]
            cropped_block = block_seg[bbox[0]:bbox[3],
                                      bbox[1]:bbox[4],
                                      bbox[2]:bbox[5]]
            change_vox = list(np.nonzero(cropped_block==t_id))
            change_vox = tuple([cv + bbox[c_id] for c_id, cv in enumerate(change_vox)])
            # update label
            block_seg[change_vox] = obj_id
            # update bbox dict
            tbbox = bbox_dict[t_id] # bbox of tracked object
            bbox_new = (min()
                        )
            bbox_dict.update({obj_id: bbox_dict.pop(t_id)})
        return track_success, block_seg, bbox_dict, iou

    obj_id = 3628
    bbox = bbox_dict[obj_id]
    print "Tracking object %d outside bbox: "%(obj_id), bbox
    check_lower = range(bbox[0]+1, 1, -zstride)
    check_higher = range(bbox[3]-1, n_slices-1, zstride)
    print "Checking slices:" check_lower, check_higher
    for sslice_id in check_lower:
        sslice = block_seg[sslice_id,:,:]
        tslice = block_seg[sslice_id-zstride,:,:]
        print "On slice: %d"%sslice_id
        block_seg, bbox_dict = track_and_relabel(sslice, tslice, obj_id, 
                                                block_seg, bbox_dict)
    for sslice_id in check_higher:
        sslice = block_seg[sslice_id,:,:]
        tslice = block_seg[sslice_id+zstride,:,:]
        print "On slice: %d"%sslice_id
        print "On slice: %d"%sslice_id
        track_results = track_and_relabel(sslice, tslice, obj_id, 
                                          block_seg, bbox_dic, iou_thresh)
        (track_success, block_seg, bbox_dict, iou) = track_results
        

[0, 202, 263, 36, 212, 269]
[] [35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58]
35
(array([3659], dtype=uint32), array([12]), array([47]))
36
(array([], dtype=uint32), array([], dtype=int64), array([], dtype=int64))
37
(array([], dtype=uint32), array([], dtype=int64), array([], dtype=int64))
38
(array([], dtype=uint32), array([], dtype=int64), array([], dtype=int64))
39
(array([], dtype=uint32), array([], dtype=int64), array([], dtype=int64))
40
(array([], dtype=uint32), array([], dtype=int64), array([], dtype=int64))
41
(array([], dtype=uint32), array([], dtype=int64), array([], dtype=int64))
42
(array([], dtype=uint32), array([], dtype=int64), array([], dtype=int64))
43
(array([], dtype=uint32), array([], dtype=int64), array([], dtype=int64))
44
(array([], dtype=uint32), array([], dtype=int64), array([], dtype=int64))
45
(array([], dtype=uint32), array([], dtype=int64), array([], dtype=int64))
46
(array([], dtype=uint32), array([], dtyp

In [ ]:
obj_id = 3628
zstride = 1